In [1]:
#! /usr/bin/python
# -*- coding: utf-8 -*-

In [2]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pyodbc
import textwrap
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
# 总访问量 - channel*income
sql1 = textwrap.dedent("""
    SELECT r.userid
          ,r.channel
          ,IIF(u.consumption>10000,'high','low') AS income
          ,COUNT(r.url) AS [Number of Visits]
    FROM [data].[dbo].[health_records] r
    JOIN [data].[dbo].[user] u
    ON r.userid = u.userid
    GROUP BY r.userid, r.channel, u.consumption
    HAVING r.channel IS NOT NULL AND u.consumption IS NOT NULL
    ORDER BY r.userid
""")
# 总访问量 - platform*income
sql2 = textwrap.dedent("""
    SELECT r.userid
          ,r.platform
          ,IIF(u.consumption>10000,'high','low') AS income
          ,COUNT(r.url) AS [Number of Visits]
    FROM [data].[dbo].[health_records] r
    JOIN [data].[dbo].[user] u
    ON r.userid = u.userid
    GROUP BY r.userid, r.platform, u.consumption
    HAVING r.platform IS NOT NULL AND u.consumption IS NOT NULL
    ORDER BY r.userid
""")
# 总访问量 - gender*income
sql3 = textwrap.dedent("""
    SELECT r.userid
          ,u.gender
          ,IIF(u.consumption>10000,'high','low') AS income
          ,COUNT(r.url) AS [Number of Visits]
    FROM [data].[dbo].[health_records] r
    JOIN [data].[dbo].[user] u
    ON r.userid = u.userid
    GROUP BY r.userid, u.gender, u.consumption
    HAVING u.gender IS NOT NULL AND u.consumption IS NOT NULL
    ORDER BY r.userid
""")
# 总访问量 - platform*gender
sql4 = textwrap.dedent("""
    SELECT r.userid
          ,r.platform
          ,u.gender
          ,COUNT(r.url) AS [Number of Visits]
    FROM [data].[dbo].[health_records] r
    JOIN [data].[dbo].[user] u
    ON r.userid = u.userid
    GROUP BY r.userid, r.platform, u.gender
    HAVING r.platform IS NOT NULL AND u.gender IS NOT NULL
    ORDER BY r.userid
""")
# 总访问量 - channel*gender
sql5 = textwrap.dedent("""
    SELECT r.userid
          ,r.channel
          ,u.gender
          ,COUNT(r.url) AS [Number of Visits]
    FROM [data].[dbo].[health_records] r
    JOIN [data].[dbo].[user] u
    ON r.userid = u.userid
    GROUP BY r.userid, r.channel, u.gender
    HAVING r.channel IS NOT NULL AND u.gender IS NOT NULL
    ORDER BY r.userid
""")
# 使用强度 - all
sql6 = textwrap.dedent("""
    SELECT r.userid
          ,r.platform
          ,r.channel
          ,COUNT(r.url) AS [Use Intensity]
          ,u.gender
          ,IIF(u.consumption>10000,'high','low') AS income
    FROM [data].[dbo].[health_records] r
    JOIN [data].[dbo].[user] u
    ON r.userid = u.userid
    GROUP BY r.userid, r.date, r.platform, r.channel, u.gender, u.consumption
    HAVING r.platform IS NOT NULL AND r.channel IS NOT NULL AND u.gender IS NOT NULL AND u.consumption IS NOT NULL
    ORDER BY r.userid, r.date
""")

In [4]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=localhost;DATABASE=data;')

cursor = cnxn.cursor()

In [5]:
# 执行sql，并将结果加载为dataframe
def load_data(sql):
    rows = cursor.execute(sql).fetchall()
    return pd.DataFrame( [[ij for ij in i] for i in rows] )

In [6]:
# 对数据做幂次变换
def power_trans(df):
    df['yt'], _ = stats.boxcox(df[3])
    return df

In [7]:
def analysis(sql,c1,c2):
    df = load_data(sql)
    df = power_trans(df)
    locals()[c1] = df[1]
    locals()[c2] = df[2]
    yt = df['yt']
    formula = 'yt~C({c1})+C({c2})+C({c1}):C({c2})'.format(c1=c1,c2=c2)
    anova_results = sm.stats.anova_lm(ols(formula,df).fit(),typ=1)
    print(anova_results)
    return None

In [8]:
# 使用强度ANOVA
df = load_data(sql6)
df = power_trans(df)
platform = df[1]
channel = df[2]
gender = df[4]
income = df[5]
yt = df['yt']
formula = """
yt~C(platform)+C(channel)+C(gender)+C(income)
  +C(platform):C(channel)+C(platform):C(gender)+C(platform):C(income)+C(channel):C(gender)+C(channel):C(income)+C(gender):C(income)
  +C(platform):C(channel):C(gender)+C(platform):C(channel):C(income)+C(platform):C(income):C(gender)+C(income):C(channel):C(gender)
  +C(platform):C(channel):C(gender):C(income)"""
anova_results = sm.stats.anova_lm(ols(formula,df).fit(),typ=1)
print(anova_results)

                                                df       sum_sq     mean_sq  \
C(platform)                                    1.0   517.689546  517.689546   
C(channel)                                     1.0    70.738192   70.738192   
C(gender)                                      1.0     4.977230    4.977230   
C(income)                                      1.0     5.720823    5.720823   
C(platform):C(channel)                         1.0     4.721865    4.721865   
C(platform):C(gender)                          1.0     1.088443    1.088443   
C(platform):C(income)                          1.0    48.393321   48.393321   
C(channel):C(gender)                           1.0     2.931171    2.931171   
C(channel):C(income)                           1.0     7.315779    7.315779   
C(gender):C(income)                            1.0    81.699720   81.699720   
C(platform):C(channel):C(gender)               1.0     2.258860    2.258860   
C(platform):C(channel):C(income)               1.0  

In [9]:
analysis(sql1,'channel','income')

                         df       sum_sq     mean_sq          F        PR(>F)
C(channel)              1.0   135.741412  135.741412  29.631075  1.050536e-07
C(income)               1.0     0.237820    0.237820   0.051914  8.199132e-01
C(channel):C(income)    1.0     0.139685    0.139685   0.030492  8.614905e-01
Residual              316.0  1447.611534    4.581049        NaN           NaN


In [10]:
analysis(sql2,'platform','income')

                          df       sum_sq    mean_sq         F    PR(>F)
C(platform)              1.0    31.729551  31.729551  5.933395  0.015420
C(income)                1.0     0.179808   0.179808  0.033624  0.854629
C(platform):C(income)    1.0    33.163544  33.163544  6.201550  0.013287
Residual               310.0  1657.762816   5.347622       NaN       NaN


In [11]:
analysis(sql3,'gender','income')

                        df       sum_sq   mean_sq         F    PR(>F)
C(gender)              1.0     0.126911  0.126911  0.025218  0.873931
C(income)              1.0     0.055511  0.055511  0.011030  0.916426
C(gender):C(income)    1.0     2.862115  2.862115  0.568726  0.451357
Residual             300.0  1509.750951  5.032503       NaN       NaN


In [12]:
analysis(sql4,'platform','gender')

                          df       sum_sq    mean_sq         F    PR(>F)
C(platform)              1.0    31.729551  31.729551  5.844744  0.016200
C(gender)                1.0     0.825770   0.825770  0.152111  0.696794
C(platform):C(gender)    1.0     7.373363   7.373363  1.358211  0.244744
Residual               310.0  1682.907036   5.428732       NaN       NaN


In [13]:
analysis(sql5,'channel','gender')

                         df       sum_sq     mean_sq          F        PR(>F)
C(channel)              1.0   135.741412  135.741412  29.634110  1.049033e-07
C(gender)               1.0     0.108247    0.108247   0.023632  8.779236e-01
C(channel):C(gender)    1.0     0.417508    0.417508   0.091147  7.629222e-01
Residual              316.0  1447.463284    4.580580        NaN           NaN
